In [4]:
import requests
import time
import pandas as pd 
import numpy as np
from datetime import datetime 
import time 
from datetime import timedelta

In [15]:
# URL base de la API CoinGecko
url = 'https://api.coingecko.com/api/v3'

# Obtengo una lista de criptomonedas y sus IDs para despues solicitar a la API los datos historicos 
coins_id_list_url = f'{url}/coins/list'
response = requests.get(coins_id_list_url)

if response.status_code == 200:
    coins_id_list = response.json()
    # Creo una lista de los IDs de las 10 criptomonedas más relevantes
    relevant_crypto_ids = ['bitcoin', 'ethereum', 'cardano', 'binancecoin', 'solana', 'ripple', 'polkadot', 'dogecoin', 'avalanche-2', 'algorand']

    # Filtro las criptos relevantes y obtengo sus IDs
    relevant_ids = [coin['id'] for coin in coins_id_list if coin['id'] in relevant_crypto_ids]
else:
    print(f'Error al obtener la lista de criptomonedas. Código de estado: {response.status_code}')

In [17]:
# Creo un diccionario vacio 
histori_data_dict = {}

# Obtengo los datos históricos para cada criptomoneda
for crypto_id in relevant_ids:
    endpoint = f'/coins/{crypto_id}/market_chart'
    intervalo_dias = 30  # Intervalo de días para cada solicitud
    today = datetime.now()
    params = {'vs_currency': 'usd', 'days': intervalo_dias}
    
    data_list = []

    while today.year > 2021:  # Obtengo datos desde 2022 hasta ahora
        response = requests.get(url + endpoint, params=params)
        
        if response.status_code == 200:
            historical_data = response.json()
            for entry in historical_data['prices']:
                timestamp = entry[0]
                price = entry[1]
                volume = next((vol[1] for vol in historical_data['total_volumes'] if vol[0] == timestamp), None)
                market_cap = next((cap[1] for cap in historical_data['market_caps'] if cap[0] == timestamp), None)
                data_list.append({
                    'Timestamp': datetime.fromtimestamp(timestamp / 1000),
                    'Price': price,
                    'Volume': volume,
                    'MarketCap': market_cap
                })
            # Agrego un retraso de 6 segundos entre las solicitudes para respetar el límite de 10 llamadas por minuto
            time.sleep(6)
        
        else:
            print(f'Error al obtener datos históricos para {crypto_id}. Código de estado: {response.status_code}')
        
        today -= timedelta(days=intervalo_dias)
        params['days'] = intervalo_dias
    
    # Guardo datos obtenidos en el diccionario
    histori_data_dict[crypto_id] = data_list


In [18]:
histori_data_dict

{'algorand': [{'Timestamp': datetime.datetime(2023, 7, 18, 16, 0, 46, 345000),
   'Price': 0.11267474876720505,
   'Volume': 57626524.91605874,
   'MarketCap': 879517791.0026734},
  {'Timestamp': datetime.datetime(2023, 7, 18, 17, 1, 9, 359000),
   'Price': 0.11238932815013356,
   'Volume': 77990517.54192829,
   'MarketCap': 873254804.5112562},
  {'Timestamp': datetime.datetime(2023, 7, 18, 18, 0, 4, 524000),
   'Price': 0.11270766718757474,
   'Volume': 60978653.14594055,
   'MarketCap': 874811877.4062734},
  {'Timestamp': datetime.datetime(2023, 7, 18, 19, 0, 39, 383000),
   'Price': 0.11334818066482145,
   'Volume': 56230928.166754656,
   'MarketCap': 881864297.7211},
  {'Timestamp': datetime.datetime(2023, 7, 18, 20, 1, 29, 471000),
   'Price': 0.1138423191811754,
   'Volume': 54495358.413052335,
   'MarketCap': 882632758.8593277},
  {'Timestamp': datetime.datetime(2023, 7, 18, 21, 0, 43, 477000),
   'Price': 0.11371268848155427,
   'Volume': 54477900.00554263,
   'MarketCap': 8821

In [22]:
# Creo DataFrames individuales por cada moneda
data_frames = {}
for crypto_id, data_list in histori_data_dict.items():
    df = pd.DataFrame(data_list)
    data_frames[crypto_id] = df

In [25]:
# Separo los df por cada moneda
df_algorand = data_frames['algorand']
df_avalanche = data_frames['avalanche-2']
df_dogecoin = data_frames['dogecoin']
df_polkadot = data_frames['polkadot']
df_ripple = data_frames['ripple']
df_solana = data_frames['solana']
df_binancecoin = data_frames['binancecoin']
df_cardano = data_frames['cardano']
df_ethereum = data_frames['ethereum']
df_bitcoin = data_frames['bitcoin']

In [28]:
# Exporto df para utilizarlo en csv
df_algorand.to_csv('df_algorand', index = False)
df_avalanche.to_csv('df_avalanche', index = False)
df_dogecoin.to_csv('df_dogecoin', index = False)
df_polkadot.to_csv('df_polkadot', index = False)
df_ripple.to_csv('df_ripple', index = False)
df_solana.to_csv('df_solana', index = False)
df_binancecoin.to_csv('df_binancecoin', index = False)
df_cardano.to_csv('df_cardano', index = False)
df_ethereum.to_csv('df_ethereum', index = False)
df_bitcoin.to_csv('df_bitcoin', index = False)